In [1]:
import os

In [2]:
!pwd

/Users/andytwang/Desktop/USF/side_project/chicken_disease/Disease-Classification/research


In [3]:
os.chdir("../")

In [4]:
!pwd

/Users/andytwang/Desktop/USF/side_project/chicken_disease/Disease-Classification


In [7]:
from dataclasses import dataclass
from pathlib import Path




In [10]:
@dataclass(frozen=True)
class PrepareBaseModelConfig():
    root_dir:Path
    base_model_path:Path
    updated_base_model_path:Path
    params_image_size: list
    params_learning_rate: float
    params_include_top: bool
    params_weights: str
    params_classes: int

In [11]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [13]:
class ConfigurationManager():
    def __init__(
        self, 
        config_path = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_path)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])
    
    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model

        create_directories([config.root_dir])

        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            updated_base_model_path=Path(config.updated_base_model_path),
            params_image_size=self.params.IMAGE_SIZE,
            params_learning_rate=self.params.LEARNING_RATE,
            params_include_top=self.params.INCLUDE_TOP,
            params_weights=self.params.WEIGHTS,
            params_classes=self.params.CLASSES            
        )
        
        return prepare_base_model_config



In [27]:
import os
import numpy as np
import urllib.request as request
from zipfile import ZipFile

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models

In [33]:
class PrepareBaseModel():
    def __init__(self, config:PrepareBaseModelConfig):
        self.config = config
        self.model = None
        self.full_model = None


    def get_base_model(self):
        self.model = models.vgg16(
            pretrained=(self.config.params_weights)
        )
        if not self.config.params_include_top:
            self.model = nn.Sequential(*list(self.model.children())[:-1])
        self.save_model(self.config.base_model_path, self.model)


    @staticmethod
    def _prepare_full_model(model, classes, freeze_all, freeze_till, learning_rate):
        if freeze_all:
            for param in model.parameters():
                param.requires_grad = False
        elif (freeze_till is not None) and (freeze_till>0):
            ct = 0
            for child in model.child:
                ct += 1
                if ct < freeze_till:
                    for param in model.parameters():
                        param.requires_grad = False
        
        num_flat_features = lambda x: int(np.prod(x.size()[1:]))
        with torch.no_grad():
            sample_tensor = torch.randn(1, 3, 224, 224)
            out = model(sample_tensor)
            in_feature = num_flat_features(out)

        full_model = nn.Sequential(
            model,
            nn.Flatten(),
            nn.Linear(in_features=in_feature, out_features=classes),
            nn.Softmax(dim=1)
        )

        criterion = nn.CrossEntropyLoss()
        optimizer = optim.SGD(full_model.parameters(), lr=learning_rate)
        print(full_model)

        return full_model, criterion, optimizer
    
    def update_base_model(self):
        self.full_model, criterion, optimizer = self._prepare_full_model(
            model=self.model,
            classes=self.config.params_classes, 
            freeze_all=True, 
            freeze_till=None, 
            learning_rate=self.config.params_learning_rate
        )

        self.save_model(path=self.config.updated_base_model_path, model=self.full_model)

    @staticmethod
    def save_model(path:Path, model:nn.Module):
        torch.save(model.state_dict(), path)

In [34]:
try:
    config = ConfigurationManager()
    prepare_base_model_config = config.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.update_base_model()
except Exception as e:
    raise e
    

[2023-08-20 02:10:17,830: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-08-20 02:10:17,834: INFO: common: yaml file: params.yaml loaded successfully]
[2023-08-20 02:10:17,836: INFO: common: created directory at: artifacts]
[2023-08-20 02:10:17,837: INFO: common: created directory at: artifacts/prepare_base_model]
Sequential(
  (0): Sequential(
    (0): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU(inplace=True)
      (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (6): ReLU(inplace=True)
      (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): ReLU(inplace=True)
      (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      